# 章節 12：證據

## 12.4 後驗分佈


下圖為 Alice 和 Bob 依據各自考試分數更新的 p_correct 後驗分佈。我們可以看到他們有重疊，有可能 Bob 的 p_correct 可能更高，但似乎機會很低。

In [1]:
#TODO

回到初始問題「說明 Alice 準備的比 Bob 好的證據有多強烈？」我們可以透過 p_correct 的後驗分佈回答此問題。

用貝葉斯檢驗的方式來表示問題，這邊定義了兩個假設：
1. Alice 的 p_correct 高於 Bob
2. Bob 的 p_correct 高於 Alice

計算 A 的似然性，我們可以列舉所有後驗分佈的配對並且加總所有 Alice 的 p_correct 高於 Bob 的機率。我們已經有個方法做這件事 thinkbayes.PmfProbGreater。

所以我們定義一個 Suite 子類別 TopLevel 來計算 A 和 B 的後驗機率：

<pre>
class TopLevel(thinkbayes.Suite):

    def Update(self, data):
        a_sat, b_sat = data
        a_like = thinkbayes.PmfProbGreater(a_sat, b_sat)
        b_like = thinkbayes.PmfProbLess(a_sat, b_sat)
        c_like = thinkbayes.PmfProbEqual(a_sat, b_sat)
        a_like += c_like / 2
        b_like += c_like / 2
        self.Mult('A', a_like)
        self.Mult('B', b_like)
        self.Normalize()
</pre>

通常當我們定義一個新的 Suite，我們繼承 Update 方法並且提供 Likelihood 方法。但這邊複寫 Update 方法，因為同時更新兩個假設比較容易。

傳進 Update 方法的 data 是 Sat 物件，代表 p_correct 的後驗分佈。

a_like 是 Alice 較大的 p_correct 總和；b_like 是 Bob 較大的 p_correct 總和。c_like 是兩者「相等」，但是這種相等性取決於建構 p_correct 的離散數列的粗細。如果我們用多點數值，c_like 會很小，若極端一點 c_like 是連續的，則 c_like 為零。所以這邊將 c_like 當作一個錯誤的四捨五入並且平均分配在 a_like 和 b_like。

以下是創造 TopLevel 物件並且更新它： 
<pre>
    exam = Exam()
    a_sat = Sat(exam, 780)
    b_sat = Sat(exam, 740)
    top = TopLevel('AB')
    top.Update((a_sat, b_sat))
    top.Print()
</pre>

A 的似然性是 0.79 而 B 的似然性為 0.21。似然性比例（貝葉斯因子）為 3.8，這表示測試分數證明 Alice 在回答 SAT 問題時比 Bob 好的程度。如果我們相信在看到測驗分數之前 A 和 B 是相同程度，但在測驗分數出來後，我們ㄧ
If we believed, before seeing the test scores, that A and B were equally likely, then after seeing the scores we should believe that the probability of A is 79%, which means there is still a 21% chance that Bob is actually better prepared.